In [34]:
import pandas as pd
from sklearn.model_selection import train_test_split
import numpy as np

df = pd.read_parquet("gros_data.parquet")
df.head()

,actors,critics_score,date,directors,editor,genre,langage,length,nationality,viewers_score,us_first_week_boxoffice,french_first_week_boxoffice,french_visa,title,vo_title,url
0,"[Arieh Worthalter, Arthur Harari, Stéphan Guér...",4.4,2023-09-27,[Cédric Kahn],Ad Vitam,"[Policier, Drame, Historique, Judiciaire]",[Français],116.0,[France],3.9,0.0,110011.0,157303,Le Procès Goldman,None,/article/fichearticle_gen_carticle=1000093547....
1,"[Monica Bellucci, Vincent Cassel, Albert Dupon...",3.5,2020-08-26,[Gaspar Noé],Carlotta Films,"[Drame, Thriller]","[Anglais, Français, Italien, Espagnol]",90.0,[France],NaN,0.0,2905.0,153336,Irréversible - Inversion Intégrale,None,/article/fichearticle_gen_carticle=18692477.html
2,[Thom Hoffman],2.8,2024-08-07,"[Richard Claus, Karsten Kiilerich]",Le Pacte,"[Aventure, Animation, Comédie, Famille]",[Néerlandais],84.0,"[Danemark, France, Allemagne, Pays-Bas]",3.0,0.0,39119.0,160622,Petit Panda en Afrique,Panda Bear in Africa,/article/fichearticle_gen_carticle=1000096064....
3,"[Lou de Laâge, Raphaël Personnaz, Isabelle Car...",3.3,2022-12-21,[Olivier Treiner],SND,[Drame],[Français],120.0,[France],3.8,0.0,64039.0,152607,Le Tourbillon de la vie,None,/article/fichearticle_gen_carticle=1000006465....
4,"[Michael B. Jordan, Jamie Foxx, Brie Larson, R...",3.0,2020-01-29,[Destin Daniel Cretton],Warner Bros. France,"[Biopic, Drame]",[Anglais],137.0,[U.S.A.],4.1,9713228.0,113153.0,152118,La Voie de la justice,Just Mercy,/video/player_gen_cmedia=19586793&cfilm=239735...


In [35]:
df_afluence_filled = pd.read_parquet("affluence_docker.parquet")
df_afluence_filled.head()

,box_office,is_prediction
month,,
1989-01-01,9.822903,1.0
1989-02-01,11.884946,1.0
1989-03-01,11.417790,1.0
1989-04-01,10.506531,1.0
1989-05-01,9.071718,1.0


In [36]:
df_actors = pd.read_parquet("affluence_docker.parquet")
df_actors.head()

,box_office,is_prediction
month,,
1989-01-01,9.822903,1.0
1989-02-01,11.884946,1.0
1989-03-01,11.417790,1.0
1989-04-01,10.506531,1.0
1989-05-01,9.071718,1.0


In [37]:
def put_national_affluence(x, filled = False):
    date = pd.to_datetime(f"{x.year}-{x.month:02d}-01")
    if filled:
        return df_afluence_filled.loc[date, 'box_office'] if date in df_afluence_filled.index else None
    else:
        return df_afluence.loc[date, 'box_office'] if date in df_afluence.index else None
    
def score_actors(x, method="boxoffice_average"):
    max_score = 0
    scores = []
    sum_score = 0
    for actor in x:
        if actor.replace(" ", "").replace("-", "").replace("_", "").strip().lower() in df_actors.index :
            max_score = max(max_score, df_actors.loc[actor.replace(" ", "").replace("-", "").replace("_", "").strip().lower(), method])
            scores.append(df_actors.loc[actor.replace(" ", "").replace("-", "").replace("_", "").strip().lower(), method])
            sum_score += df_actors.loc[actor.replace(" ", "").replace("-", "").replace("_", "").strip().lower(), method]
    if len(scores) == 0:
        mean_scores = 0
    else:
        mean_scores = np.mean(scores)
    return {"max_score":max_score, "mean_scores":mean_scores, "sum_score":sum_score}

In [38]:
for col in ['genre', 'langage', 'nationality', 'actors']:
    df[col] = df[col].mask(df[col].isna(), ['no value'])
df['national_affluence'] = df['date'].apply(lambda x : put_national_affluence(x,True))
actor_scores = df['actors'].apply(lambda x : score_actors(x)) 
df['french_prod'] = df['nationality'].apply(lambda x: 1 if "France" in x else 0)
df['usa_prod'] = df['nationality'].apply(lambda x: 1 if "U.S.A." in x else 0)
df['french_langage'] = df['langage'].apply(lambda x: 1 if "Français" in x else 0)
df['english_langage'] = df['langage'].apply(lambda x: 1 if "Anglais" in x else 0)
df['number_actors'] = df['actors'].apply(lambda x: len(x))
df['max_average_actor'] = actor_scores.apply(lambda x : x["max_score"])
df['sum_score_actor'] = actor_scores.apply(lambda x : x["sum_score"])
df['mean_average_actor'] = actor_scores.apply(lambda x : x["mean_scores"])

In [39]:
df.columns

Index(['actors', 'critics_score', 'date', 'directors', 'editor', 'genre',
       'langage', 'length', 'nationality', 'viewers_score',
       'us_first_week_boxoffice', 'french_first_week_boxoffice', 'french_visa',
       'title', 'vo_title', 'url', 'national_affluence', 'french_prod',
       'usa_prod', 'french_langage', 'english_langage', 'number_actors',
       'max_average_actor', 'sum_score_actor', 'mean_average_actor'],
      dtype='object')

In [40]:
df.dtypes

actors                                 object
critics_score                         float64
date                           datetime64[ns]
directors                              object
editor                                 object
genre                                  object
langage                                object
length                                float64
nationality                            object
viewers_score                         float64
us_first_week_boxoffice               float64
french_first_week_boxoffice           float64
french_visa                            object
title                                  object
vo_title                               object
url                                    object
national_affluence                    float64
french_prod                             int64
usa_prod                                int64
french_langage                          int64
english_langage                         int64
number_actors                     

In [41]:
df["max_average_actor"].isna().value_counts()

max_average_actor
False    9151
Name: count, dtype: int64

In [ ]:
for col in ['genre', 'langage', 'directors', 'nationality']:
    col_res = []
    col_boxoffice_means = {}
    for col_list in df[col]:
        col_res.extend(col_list)
    unique_col = list(set(col_res))
    for item in unique_col:
        df_mask = df[df[col].apply(lambda x : item in x )]
        mean_value = df_mask['french_first_week_boxoffice'].mean()
        col_boxoffice_means[item] = mean_value
    df[f"{col}_target_transform"] = df[col].apply(lambda x : [col_boxoffice_means[s] for s in x])
    df[f"{col}_target_transform"] = df[f"{col}_target_transform"].apply(lambda x : sorted(x,reverse=True))
    df[f"{col}_first_target_transform"] = df[f"{col}_target_transform"].apply(lambda x : x[0] if len(x)>0 else 0)
    df[f"{col}_second_target_transform"] = df[f"{col}_target_transform"].apply(lambda x : x[1] if len(x)>1 else 0)
    df[f"{col}_third_target_transform"] = df[f"{col}_target_transform"].apply(lambda x : x[2] if len(x)>2 else 0)


0       189607.906250
1            0.000000
2       537705.826464
3            0.000000
4            0.000000
            ...      
9153         0.000000
9154         0.000000
9155         0.000000
9156         0.000000
9157         0.000000
Name: genre_third_target_transform, Length: 9151, dtype: float64
0       420527.528184
1       381754.506923
2       729750.547043
3       141083.341007
4       211960.965418
            ...      
9153    792219.491342
9154    381754.506923
9155    614841.678750
9156    381754.506923
9157    141083.341007
Name: genre_first_target_transform, Length: 9151, dtype: float64


In [9]:
import pandas as pd
import numpy as np
import pycaret.regression as pcr
from sklearn.model_selection import TimeSeriesSplit

# Ajout des nouvelles features
# df['month'] = df['date'].dt.month
# df['day_of_week'] = df['date'].dt.dayofweek
# df['is_summer'] = df['month'].isin([6, 7, 8]).astype(int)

# One-hot encoding pour les list
for col in ['genre', 'langage', 'directors', 'nationality']:
    col_res = []
    for col_list in df[col]:
        col_res.extend(col_list)
    unique_col = list(set(col_res))

    for truc in unique_col:
        df[f'{col}_{truc}'] = df[col].apply(lambda x: 1 if truc in x else 0)

print(df.shape)

(9151, 5950)


In [10]:
def time_based_split(df, test_years=3, test_ratio=0.5):
    """
    Crée un découpage train/test basé sur le temps.
    
    Args:
        df: DataFrame avec les données
        test_years: Nombre d'années récentes à considérer pour le découpage
        test_ratio: Proportion des films des années récentes à mettre dans le test set
    
    Returns:
        indices_train, indices_test
    """
    # Convertir en datetime si ce n'est pas déjà fait
    df["date"] = pd.to_datetime(df["date"])
    
    # Déterminer la date limite pour les dernières années
    max_date = df["date"].max()
    cutoff_date = pd.Timestamp(year=max_date.year - test_years, month=1, day=1)
    
    # Créer un masque pour les films récents
    recent_mask = df["date"] >= cutoff_date
    
    # Séparer les indices en entrainement et test
    old_indices = df[~recent_mask].index.tolist()
    recent_indices = df[recent_mask].index.tolist()
    
    # Trier les indices récents par date pour pouvoir prendre un film sur deux
    recent_sorted = sorted([(idx, df.loc[idx, "date"]) for idx in recent_indices], 
                           key=lambda x: x[1])
    recent_indices_sorted = [item[0] for item in recent_sorted]
    
    # Prendre un film sur deux dans les données récentes pour le test
    test_size = int(len(recent_indices_sorted) * test_ratio)
    
    # Option 1: Prendre aléatoirement test_ratio des films récents
    np.random.seed(42)
    test_indices = np.random.choice(recent_indices_sorted, size=test_size, replace=False)
    
    # Option 2: Prendre un film sur deux chronologiquement
    # test_indices = recent_indices_sorted[::int(1/test_ratio)] if test_ratio <= 0.5 else recent_indices_sorted

    train_indices = [idx for idx in df.index if idx not in test_indices]
    
    print(f"Découpage temporel: {len(train_indices)} films pour l'entrainement, {len(test_indices)} films pour le test")
    # print(f"Plage de dates d'entrainement: {df.loc[train_indices, "date"].min()} à {df.loc[train_indices, "date"].max()}")
    # print(f"Plage de dates de test: {df.loc[test_indices, "date"].min()} à {df.loc[test_indices, "date"].max()}")
    
    return train_indices, test_indices

In [11]:
columns_to_drop = ['critics_score', 'viewers_score', 'us_first_week_boxoffice', 
                   'french_visa', 'title', 'vo_title', 'url', 'genre', 'directors', 'langage', 'nationality', 'actors']

train_indices, test_indices = time_based_split(df, test_years=3, test_ratio=0.3)
train = df.loc[train_indices]
test = df.loc[train_indices]

# df = df.sort_values(by='date')
# split_date = df['date'].iloc[int(len(df) * 0.8)]  # 80% train, 20% test
# train = df[df['date'] < split_date]
# test = df[df['date'] >= split_date]
train.drop(columns_to_drop, axis=1, inplace=True)
test.drop(columns_to_drop, axis=1, inplace=True)
train['transformed_target'] = np.log1p(train['french_first_week_boxoffice'])
test['transformed_target'] = np.log1p(test['french_first_week_boxoffice'])

Découpage temporel: 8694 films pour l'entrainement, 457 films pour le test


In [ ]:
setup = pcr.setup(data=train,
                  target='transformed_target',
                  normalize=True,
                  transformation=True,
                  transform_target=False, 
                  date_features=['date'],
                  session_id=123)

# Comparaison des modèles
best_models = pcr.compare_models(sort='RMSE')

# Création d'un modèle de stacking (combinaison de plusieurs modèles)
stacker = pcr.create_model('stack_models')

# Tuning des hyperparamètres
tuned_model = pcr.tune_model(stacker)

# Validation finale
eval_model = pcr.evaluate_model(tuned_model)

# Prédiction sur l'ensemble de test
predictions = pcr.predict_model(tuned_model, data=test)

# Analyse des erreurs
pcr.plot_model(tuned_model, plot='residuals')
pcr.plot_model(tuned_model, plot='feature')

,Description,Value
0,Session id,123
1,Target,transformed_target
2,Target type,Regression
3,Original data shape,"(8694, 5939)"
4,Transformed data shape,"(8694, 5941)"
5,Transformed train set shape,"(6085, 5941)"
6,Transformed test set shape,"(2609, 5941)"
7,Numeric features,5936
8,Date features,1
9,Categorical features,1


,,
,,
Initiated,. . . . . . . . . . . . . . . . . .,11:04:18
Status,. . . . . . . . . . . . . . . . . .,Fitting 10 Folds
Estimator,. . . . . . . . . . . . . . . . . .,Ridge Regression


,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE,TT (Sec)
lasso,Lasso Regression,0.8210,1.0939,1.0451,0.8261,0.1268,0.1036,11.1110
lr,Linear Regression,2661925779000.2485,175863450273431951272574976.0000,13059125030560.9121,-28030436601603861143093248.0000,16.0281,314885123452.4015,83.1800


Processing:   0%|          | 0/81 [00:00<?, ?it/s]

In [ ]:
# import pandas as pd
# import numpy as np
# from pycaret.regression import *
# from sklearn.base import BaseEstimator, TransformerMixin

# # Vos transformateurs personnalisés
# class MultiLabelBinarizerTransformer(BaseEstimator, TransformerMixin):
#     def __init__(self):
#         self.mlbs = {}  # Stocke un MultiLabelBinarizer pour chaque colonne
    
#     def fit(self, X, y=None):
#         for col in X.columns:
#             self.mlbs[col] = MultiLabelBinarizer()
#             self.mlbs[col].fit(X[col])
#         return self
    
#     def transform(self, X):
#         transformed_list = []
#         for col in X.columns:
#             transformed = self.mlbs[col].transform(X[col])
#             new_columns = [f"{col}_{label}" for label in self.mlbs[col].classes_]
#             transformed_list.append(pd.DataFrame(transformed, columns=new_columns, index=X.index))
#         return pd.concat(transformed_list, axis=1)
    
#     def get_feature_names_out(self, input_features=None):
#         # Collecter tous les noms de colonnes de sortie
#         feature_names = []
#         for col in self.mlbs.keys():
#             feature_names.extend([f"{col}_{label}" for label in self.mlbs[col].classes_])
#         return np.array(feature_names)

# class CustomDateTransformer(BaseEstimator, TransformerMixin):
#     def __init__(self):
#         self.feature_names_out = ['year', 'month', 'day', 'dayofweek']
        
#     def fit(self, X, y=None):
#         return self
        
#     def transform(self, X):
#         # Assumant que X est un DataFrame avec une colonne 'date'
#         result = pd.DataFrame({
#             'year': X.dt.year,
#             'month': X.dt.month,
#             'day': X.dt.day,
#             'dayofweek': X.dt.dayofweek
#         })
#         return result
    
#     def get_feature_names_out(self, input_features=None):
#         return np.array(self.feature_names_out)

# # Fonction pour préparer les données et les transformer avec vos transformateurs personnalisés
# def prepare_data_for_pycaret(df, list_categorical_features=None):
#     """
#     Prépare les données en appliquant les transformateurs personnalisés
#     avant de les passer à PyCaret
    
#     Args:
#         df: DataFrame à transformer
#         list_categorical_features: Liste des colonnes contenant des listes de catégories
    
#     Returns:
#         DataFrame transformé
#     """
#     df_copy = df.copy()
    
#     # Appliquer le transformateur de date si une colonne 'date' existe
#     if 'date' in df_copy.columns:
#         date_transformer = CustomDateTransformer()
#         date_features = date_transformer.transform(df_copy['date'])
        
#         # Supprimer la colonne date originale et ajouter les nouvelles colonnes
#         df_copy = df_copy.drop('date', axis=1)
#         df_copy = pd.concat([df_copy, date_features], axis=1)
    
#     # Appliquer le MultiLabelBinarizer pour les colonnes de listes catégorielles
#     if list_categorical_features:
#         list_transformer = MultiLabelBinarizerTransformer()
#         list_features_df = list_transformer.fit_transform(df_copy[list_categorical_features])
        
#         # Supprimer les colonnes originales et ajouter les nouvelles colonnes
#         df_copy = df_copy.drop(list_categorical_features, axis=1)
#         df_copy = pd.concat([df_copy, list_features_df], axis=1)
    
#     return df_copy

# # Exemple d'utilisation avec PyCaret
# def run_pycaret_regression(data, target_column, list_categorical_features=None):
#     """
#     Configuration et exécution d'un modèle de régression avec PyCaret
    
#     Args:
#         data: DataFrame avec toutes les colonnes
#         target_column: Nom de la colonne cible
#         list_categorical_features: Liste des colonnes contenant des listes de catégories
#     """
#     # Préparer les données
#     prepared_data = prepare_data_for_pycaret(data, list_categorical_features)
    
#     # Initialiser l'environnement PyCaret avec un ensemble réduit de paramètres
#     s = setup(
#         data=prepared_data,
#         target=target_column,
#         session_id=42,
#         normalize=True,           # Normalisation des variables numériques
#         transformation=True,      # Transformations automatiques pour la correction de la skewness
#         ignore_features=None,     # Colonnes à ignorer
#         categorical_features=None, # Colonnes catégorielles standards
#         numeric_features=None,    # Colonnes numériques
#         date_features=None,       # Colonnes de date (déjà transformées)
#         imputation_type='simple', # Méthode d'imputation
#         numeric_imputation='median', # Valeur pour l'imputation des valeurs numériques
#         categorical_imputation='mode', # Valeur pour l'imputation des valeurs catégorielles
#         remove_multicollinearity=True, # Supprimer la multicolinéarité
#         multicollinearity_threshold=0.9, # Seuil de multicolinéarité
#         n_jobs=-1,               # Nombre de processeurs à utiliser (-1 pour tous)
#         verbose=True              # Afficher les détails
#     )
    
#     # Comparer tous les modèles
#     best_models = compare_models(
#         sort='RMSE',              # Trier par RMSE
#         n_select=3,               # Sélectionner les 3 meilleurs modèles
#         fold=5                    # 5-fold cross-validation
#     )
    
#     # Afficher le meilleur modèle
#     print("\nMeilleur modèle:")
#     print(best_models[0])
    
#     # Créer un modèle d'ensemble avec les meilleurs modèles
#     ensemble_model = ensemble_model(best_models)
    
#     # Évaluer le modèle d'ensemble
#     evaluation = evaluate_model(ensemble_model)
    
#     # Affiner le meilleur modèle
#     tuned_model = tune_model(
#         best_models[0],
#         optimize='RMSE',
#         n_iter=50
#     )
    
#     # Évaluer le modèle optimisé
#     tuned_evaluation = evaluate_model(tuned_model)
    
#     # Retourner les modèles
#     return {
#         'setup': s,
#         'best_models': best_models,
#         'ensemble_model': ensemble_model,
#         'tuned_model': tuned_model
#     }

# # Pour déployer le modèle final
# def finalize_pycaret_model(models_dict, X_train, y_train, X_test=None, y_test=None):
#     """
#     Finalise le modèle PyCaret avec les meilleures performances
#     """
#     # Finaliser le modèle (entraînement sur l'ensemble complet)
#     final_model = finalize_model(models_dict['tuned_model'])
    
#     # Sauvegarder le pipeline complet
#     save_model(final_model, 'final_pycaret_model')
    
#     # Prédictions sur les données de test si disponibles
#     if X_test is not None and y_test is not None:
#         predictions = predict_model(final_model, data=X_test)
#         print("Performance sur les données de test:")
#         print(f"RMSE: {np.sqrt(mean_squared_error(y_test, predictions['prediction']))}")
    
#     return final_model

In [ ]:
# train_data = X_train.copy()
# train_data[target] = y_train

# models = run_pycaret_regression(
#     data=train_data,
#     target_column=target,
#     list_categorical_features=list_categorical_features
# )

In [ ]:
# final_model = finalize_pycaret_model(
#     models_dict=models,
#     X_train=X_train,
#     y_train=y_train,
#     X_test=X_test,
#     y_test=y_test
# )